<img src="images/lasalle_logo.png" style="width:375px;height:110px;">

# Week 10 - Data Models and Relational SQL

### WIM250 - Introduction to Scripting Languages 
### Instructor: Ivaldo Tributino

Sources:
- Python for Everybody Exploring Data Using Python 3 by Dr. Charles R. Severance.
- [Filemaker/Many-to-many relationships](https://fmhelp.filemaker.com/help/18/fmp/en/index.html#page/FMP_Help/many-to-many-relationships.html)


## Data modeling

The real power of a relational database is when we `create multiple tables` and make `links between those tables`. The act of deciding how to break up your application data into multiple tables and establishing the relationships between the tables is called `data modeling`. The design document that shows the tables and their relationships is called a `data model`.

<img src="images/relational.png" style="width:375px;height:300px;">

For more detail on data modeling you can start with: [Relational Data Model in DBMS: Concepts, Constraints, Example](https://www.guru99.com/relational-data-model-dbms.html)


In the last week, we created a Database with only one table called `Daily` and columns: `city_name, longitude, latitude and max_tem`. For this week, we will create more tables to contain extra data such as temperatures and country of each city. For that, we need to establish a relationship with these tables.

In [ ]:
import json
daily = [json.loads(line) for line in open('daily_14.json', encoding="utf8")]

In [ ]:
# Let's take a look at this list.
daily[0]['city']

In [ ]:
daily[0]['data'][0]

In [ ]:
# Let's collect the data we are interested in.
data = []
for city in daily:
    dic = {}
    temperature = []
    dic['city'] = city['city']
    for temp in city['data']:
        dic_temp = {}
        dic_temp['dt'] = temp['dt']
        dic_temp['temp_min'] = temp['temp']['min']
        dic_temp['temp_max'] = temp['temp']['max']
        dic_temp['temp_night'] = temp['temp']['night']
        dic_temp['temp_eve'] = temp['temp']['eve']
        dic_temp['temp_morn'] = temp['temp']['morn']
        temperature.append(dic_temp)
    dic['data'] = temperature    
    data.append(dic)   

In [ ]:
data[0]

Let's create two tables named `Country` and `City`. 

We should never put the same string data in the database more than once, `Duplication of string` is a data violation. If we need the data more than once, we create a `numeric key` for the data and reference the actual data using this key. The reason is because, in practical terms, a string takes up much more space than an integer on our computer's disk and memory and takes more time for the processor to compare and sort.

The City table needs an additional column to store the numeric key associated with the row for this Countries. SQLite has a feature that automatically adds the key value for any row we insert into a table using a special type of data column **(INTEGER PRIMARY KEY)**.

```SQL
CREATE TABLE Country (id INTEGER PRIMARY KEY, country_code TEXT UNIQUE);
CREATE TABLE City (id INTEGER PRIMARY KEY, city_name TEXT, country_id INTEGER, latitude FLOAT, longitude FLOAT)
```

We also add the keyword **UNIQUE** to indicate that we will not allow SQLite to insert two rows with the same value for name.

<img src="images/tables.png" style="width:400px;height:350px;">

In [ ]:
import sqlite3

conn = sqlite3.connect('daily.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Country;
DROP TABLE IF EXISTS City;

CREATE TABLE Country (
    id             INTEGER PRIMARY KEY UNIQUE,
    country_code   TEXT UNIQUE
);


CREATE TABLE City (
    id             INTEGER PRIMARY KEY UNIQUE,
    city_name      TEXT,
    country_id     INTEGER,
    latitude       REAL,
    longitude      REAL

)''')

# Only the first 20 cities.
for dic in data[:20]:
    
    city_id = dic['city']['id']
    city_name = dic['city']['name']
    country = dic['city']['country']
    lon = dic['city']['coord']['lon']
    lat = dic['city']['coord']['lat']
  
    
    cur.execute('''INSERT OR IGNORE INTO Country (country_code) 
        VALUES ( ? )''', (country,) )
    cur.execute('SELECT id FROM Country WHERE country_code = ? ', (country, ))
    country_id = cur.fetchone()[0] # retrieve the record 
        
    cur.execute('''INSERT OR IGNORE INTO City (id, city_name, country_id, latitude, longitude) 
        VALUES ( ?, ?, ?, ?, ? )''', (city_id, city_name, country_id, lat, lon) )


conn.commit() #  force the database to really be updated

We added the **OR IGNORE** clause to our **INSERT** statement to indicate that if this particular **INSERT** would cause a violation of the “Unique” rule, the database system is allowed to ignore the **INSERT**.

### It is time to add the last table, the temperature table.

<img src="images/temp_table.png" style="width:300px;height:375px;">

In [ ]:
from datetime import datetime

conn = sqlite3.connect('daily.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Country;
DROP TABLE IF EXISTS City;
DROP TABLE IF EXISTS Temperature;

CREATE TABLE Country (
    id             INTEGER PRIMARY KEY UNIQUE,
    country_code   TEXT UNIQUE
);


CREATE TABLE City (
    id             INTEGER PRIMARY KEY UNIQUE,
    city_name      TEXT,
    country_id     INTEGER,
    latitude       FLOAT,
    longitude      FLOAT

);

CREATE TABLE Temperature (
    city_id     INTEGER,
    time        INTEGER,
    tem_min     FLOAT,
    tem_max     FLOAT,
    tem_night   FLOAT,
    tem_eve     FLOAT,
    tem_morn    FLOAT
)''')


for dic in data[:20]:

    city_id = dic['city']['id']
    city_name = dic['city']['name']
    country = dic['city']['country']
    lon = dic['city']['coord']['lon']
    lat = dic['city']['coord']['lat']
  
    
    cur.execute('''INSERT OR IGNORE INTO Country (country_code) 
        VALUES ( ? )''', (country,) )
    cur.execute('SELECT id FROM Country WHERE country_code = ? ', (country, ))
    country_id = cur.fetchone()[0] # retrieve the record 
        
    cur.execute('''INSERT OR IGNORE INTO City (id, city_name, country_id, latitude, longitude) 
        VALUES ( ?, ?, ?, ?, ? )''', (city_id, city_name, country_id, lat, lon) )
    
    for tem in dic['data']:    
        
        time = datetime.fromtimestamp(tem['dt'])
        tem_min = tem['temp_min']
        tem_max = tem['temp_max']
        tem_night = tem['temp_night']
        tem_eve = tem['temp_eve']
        tem_morn = tem['temp_morn']
        


        cur.execute('''INSERT OR REPLACE INTO Temperature
            (city_id, time, tem_min, tem_max, tem_night, tem_eve, tem_morn) 
            VALUES ( ?, ?, ?, ?, ?, ?, ?)''', 
            ( city_id, time, tem_min, tem_max, tem_night, tem_eve, tem_morn ) )

conn.commit()

## Many-to-many relationships (Student's Assignment)

A `many-to-many relationship` occurs when multiple records in a table are associated with multiple records in another table. For example, a `many-to-many relationship` exists between customers and products: customers can purchase multiple products, and products can be purchased by many customers.

Another example of a many-to-many relationship is one between students and classes. A student can register for many classes, and a class can include many students.

<img src="images/many_to_many.png" style="width:625px;height:300px;">

[<center>Many-to-many relationships</center>](https://fmhelp.filemaker.com/help/16/fmp/en/#page/FMP_Help%2Fmany-to-many-relationships.html%23)

To break the many-to-many relationship into two one-to-many relationships we use a third table, called a `join table`. Each record in a `join table` includes a match field that contains the value of the `primary keys` of the two tables it joins. (In the join table, these match fields are `foreign keys`.) 

In [ ]:
# Let's take a look at the json file that will be used in your assignment.
with open('assignment.json', 'r') as myfile:
    students = json.load(myfile)
        
students = json.loads(students)    

In [ ]:
students

In [ ]:
for st in students:
    print(st['studentId'],st['firstName'], st['lastName'])

In [ ]:
for idx, st in enumerate(students):
    print('Student' + str(idx+1))
    for course in st['courses']:
        print('Title: ', course['title'])

In [ ]:
students[6]['courses'][2]['description']

In [ ]:
' '.join(students[6]['courses'][2]['description'].replace("\n", " ").split())

# Using JOIN to retrieve data

```SQL
SELECT * FROM City JOIN Temperature ON Temperature.city_id = City.id

SELECT * FROM City JOIN Temperature ON Temperature.city_id = City.id WHERE Temperature.time = '2017-03-20 23:00:00'


SELECT  City.city_name, Temperature.tem_max  
FROM City JOIN Temperature 
ON Temperature.city_id = City.id 
WHERE Temperature.time = '2017-03-20 23:00:00'


SELECT  Country.country_code, City.city_name, Temperature.tem_max  
FROM Country JOIN City JOIN Temperature 
ON  City.country_id = Country.id AND Temperature.city_id = City.id 
WHERE Temperature.time = '2017-03-20 23:00:00'
                        

```

In [ ]:
import sqlite3

conn = sqlite3.connect('daily.sqlite')
cur = conn.cursor()

cur.execute('''SELECT  Country.country_code, City.city_name, Temperature.tem_max  
               FROM Country JOIN City JOIN Temperature 
               ON  City.country_id = Country.id AND Temperature.city_id = City.id 
               WHERE Temperature.time = ?''', ('2017-03-20 23:00:00',))
count = 0
print('city:')
for row in cur:
    if count < 20: print(row)
    count = count + 1
print(count, 'rows.')

cur.close()

## Import functions/Classes from another `.ipynb` file

In [ ]:
# You need to install ipynb package. Try: pip install ipynb 
# (see : https://stackoverflow.com/questions/20186344/importing-an-ipynb-file-from-another-ipynb-file)
from ipynb.fs.full.classes_functions import get_keys
from ipynb.fs.full.classes_functions import database

In [ ]:
get_keys(data)

In [ ]:
data_SQlite = database(data)

In [ ]:
dir(data_SQlite)

In [ ]:
type(data_SQlite.data)

In [ ]:
dir(data_SQlite.data[0])

In [ ]:
dir(data_SQlite.data[0].country)

In [ ]:
data_SQlite.data[0].country.countryInfo()

In [ ]:
dir(data_SQlite.data[0].city)

In [ ]:
data_SQlite.data[0].city.cityInfo(2)

In [ ]:
type(data_SQlite.data[0].temp)

In [ ]:
dir(data_SQlite.data[0].temp[0])

In [ ]:
data_SQlite.data[0].temp[0].tempInfo()

In [ ]:
conn = sqlite3.connect('daily_1.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript(data_SQlite.script().replace("\n", " "))

for data in data_SQlite.data:
        
    cur.execute(data.country.countryInfo()[0], data.country.countryInfo()[1])
    cur.execute('SELECT id FROM Country WHERE country_code = ? ', (data.country.countryInfo()[1][0],))
    country_id = cur.fetchone()[0] # retrieve the record 
        
    cur.execute(data.city.cityInfo(country_id)[0], data.city.cityInfo(country_id)[1])

    for temp in data.temp:

        cur.execute(temp.tempInfo()[0], temp.tempInfo()[1])

conn.commit()

In [ ]:
import sqlite3

conn = sqlite3.connect('daily_1.sqlite')
cur = conn.cursor()

cur.execute('''SELECT  Country.country_code, City.city_name, Temperature.tem_max  
               FROM Country JOIN City JOIN Temperature 
               ON  City.country_id = Country.id AND Temperature.city_id = City.id 
               WHERE Temperature.time = ?''', ('2017-03-20 23:00:00',))
count = 0
print('city:')
for row in cur:
    if count < 15: print(row)
    count = count + 1
print(count, 'rows.')

cur.close()